# Product Matching using ML

In [97]:
#import required modules
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
from sklearn.metrics.pairwise import cosine_similarity

In [142]:
#import data
flipkart_df = pd.read_csv('flipkart_com-ecommerce_sample.csv')
amz_df = pd.read_csv('amz_com-ecommerce_sample.csv', encoding='latin')

In [65]:
flipkart_df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [66]:
amz_df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [249]:
#Keeping only required columns
cols = ['product_name','retail_price','discounted_price','description']
flipkart_df = flipkart_df[cols]
amz_df = amz_df[cols]

In [413]:
#Function to find similarity between different products
#Here we have considering both title and description, as both of them are equaly important for find similarity in products
#We are also assuming that product title might be similar to reduce running time by reducing the products to consider

def get_match(title, desc):
    
    sen = [re.sub('[^A-Za-z0-9]+', ' ', title+' '+desc).strip()]
    position = []
    
    #Reducing product to consider by filtering by title
    for i in range(amz_df.shape[0]):
        if title.split(' ')[0] == amz_df['product_name'][i].split(' ')[0].lower():
            search = str(amz_df['product_name'][i]).lower()+' '+str(amz_df['description'][i]).lower()
            sen.append(re.sub('[^A-Za-z0-9]+', ' ', search).strip())
            position.append(i)

    #Encoding:
    sen_embeddings = model.encode(sen)
    
    #Calculate cosine similarity for sentence 0:
    similarity = cosine_similarity(
        [sen_embeddings[0]],
        sen_embeddings[1:]
    )
    
    #Creating a dataframe to return all similar products, as some product might have 2 or more similar products
    df = pd.DataFrame()
    for s in range(len(similarity[0])):
        try:
            if similarity[s]>=1. or similarity[s]==max(similarity):
                df = pd.concat([df,amz_df.iloc[position[s]:position[s]+1]], axis=0)
        except:
            if similarity[0][s]>=1. or similarity[0][s]==max(similarity[0]):
                df = pd.concat([df,amz_df.iloc[position[s]:position[s]+1]], axis=0)

    df = df.reset_index(drop=True)
    return df

In [414]:
#Function to find all similar products of all the items and returning a dataframe

def result_df(num):
    df = pd.DataFrame()
    f_name, f_price, f_discount, a_name, a_price, a_discount = [], [], [], [], [], []
    
    for index in range(num):
        try:
            return_df = get_match(flipkart_df['product_name'][index].lower(),flipkart_df['description'][index].lower())
        except:
            return_df = get_match(flipkart_df['product_name'][index].lower(),'nan')
        
        for i in range(return_df.shape[0]):
            f_name.append(flipkart_df['product_name'][index])
            f_price.append(flipkart_df['retail_price'][index])
            f_discount.append(flipkart_df['discounted_price'][index])
            a_name.append(return_df['product_name'][i])
            a_price.append(return_df['retail_price'][i])
            a_discount.append(return_df['discounted_price'][i])
        
    df['Product Name in Flipkart'] = f_name
    df['Retail Price in Flipkart'] = f_price
    df['Discounted Price in Flipkart'] = f_discount
    df['Product Name in Amazon'] = a_name
    df['Retail Price in Amazon'] = a_price
    df['Discounted Price in Amazon'] = a_discount
    
    df = df.reset_index(drop=True)
    return(df)

In [415]:
#Here 35 is the number of items in Flipkart DataFrame that are to matched in Amazon DataFrame
output_df = result_df(35)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [416]:
output_df

,Product Name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart,Product Name in Amazon,Retail Price in Amazon,Discounted Price in Amazon
0,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,982,438
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32143,29121
2,AW Bellies,999.0,499.0,AW Bellies,991,551
3,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,694,325
4,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,Sicons All Purpose Arnica Dog Shampoo,208,258
5,Eternal Gandhi Super Series Crystal Paper Weig...,430.0,430.0,Eternal Gandhi Super Series Crystal Paper Weig...,427,473
6,Alisha Solid Women's Cycling Shorts,1199.0,479.0,Alisha Solid Women's Cycling Shorts,1198,602
7,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32137,28664
8,"dilli bazaaar Bellies, Corporate Casuals, Casuals",699.0,349.0,"dilli bazaaar Bellies, Corporate Casuals, Casuals",682,385
9,Alisha Solid Women's Cycling Shorts,1199.0,479.0,Alisha Solid Women's Cycling Shorts,1197,542


Here we are using Sentence Transformers and using BERT, because they are very effective in large scale comparisons and nlp tasks. They are scalable, efficient and outperforms many models in semantic textual similarity tasks.

<h3>Assumptions</h3>
<br>

- Title and Description are equally important in matching products.

- Product can be filtered using title, so to reduce running time by minimizing the products that are to be consider.

- All products descriptions and titles are in one language, i.e. English